In [2]:
#import packages

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
import time
import pooch
import requests
warnings.filterwarnings('ignore')

In [68]:
df_CO = pd.read_csv('./data/COGCC wells.csv') # downloaded from COGCC all Colorado wells

In [69]:
#Extract wells only Raton Basin 
twp_list = ['31S', '32S', '33S', '34S', '35S']
range_list = ['61W', '62W', '63W', '64W', '65W', '66W', '67W', '68W']
temp = df_CO[df_CO['Township'].isin(twp_list)] 
df_Raton_CO = temp[temp['Range'].isin(range_list)] 

In [70]:
df_Raton_CO[['Facil_Type',  'Facil_Stat', 'Facil_Id']]

,Facil_Type,Facil_Stat,Facil_Id
37999,WELL,DM,217167
38000,WELL,DA,217951
38001,WELL,PA,217169
38004,WELL,DA,217172
38005,WELL,DA,217173
...,...,...,...
41972,WELL,AL,437234
41981,WELL,AL,292200
41983,WELL,DA,217952
41984,WELL,pa,217953


In [71]:
status_list = ['PR', 'PA', 'TA']# 'SU', 'SI']
df_Raton_CO_act = df_Raton_CO[df_Raton_CO['Facil_Stat'].isin(status_list)]
len(df_Raton_CO_act)

3077

In [72]:
def top_parse(text):
    tops = text.split('<!-- LOOP FOR EACH Formation WITHIN WELLBORE -->')[1].split('<!-- END Formation LOOP -->')[0]
    tops = tops.replace('\r','').replace('\n','').replace('\t','').strip()
    tops = tops.split('<tr>')
    tops = [x.strip() for x in tops if len(x) > 0]
    
    formations = {}
    for top in tops:
        cols = top.split('<td')
        cols = [x.strip() for x in cols if len(x) > 0]    
        formation = cols[0].split('color="Navy">')[1].split(' ')[0]
        depth = eval(cols[1].split('<font size="2">')[1].split('</font>')[0])
        formations[formation] = depth
    return formations

In [87]:
#FOR loop to extract all formation tops available in COGCC Raton Basin

wells_list = df_Raton_CO_act.api10.values[0:49]
#wells_list = [507106224, 507106360, 507106650, 507106111, 507160012, 507160013]
wells_tops = np.array([])
baseURL = 'https://cogcc.state.co.us/cogis/FacilityDetail.asp?facid='
tailURL = '&type=WELL'

for idx in range(len(wells_list)):
    url = baseURL + str(wells_list[idx])[1:] + tailURL
    r = requests.get(url)
    #print('URL:', url)
    #wells_tops = np.append(wells_tops,[wells_list[idx],top_parse(r.text)], axis=0)
    try:
          wells_tops = np.append(wells_tops,[wells_list[idx],top_parse(r.text)], axis=0)
    except:
        pass

In [88]:
print(wells_tops)

[]


In [80]:
#pd.DataFrame(wells_tops).to_csv('./data/6038_tops.csv')

In [83]:
tops_Raton = pd.DataFrame(wells_tops)

In [85]:
tops_Raton.head(10)

,0
0,507105004
1,{}
2,507105010
3,"{'NIOBRARA': 3736, 'GREENHORN': 4693, 'DAKOTA'..."
4,507105017
5,{}
6,507105027
7,{}
8,507105028
9,{}
